# **Phase 2 - Data Pre-processing**

In this phase, we perform all ncessary cleaning steps such as:

*   Handling Missing Values
*   Removing duplicates
*   Correcting data types
*   Creating derived columns
*   Filtering or aggregating data










In [ ]:
#load datasets

from google.colab import files    # to upload files from the computer into the Colab environment.
import zipfile
import io

# Upload ZIP file
uploaded = files.upload()

# Get the uploaded ZIP filename
zip_filename = list(uploaded.keys())[0]

# Extract ZIP file
with zipfile.ZipFile(io.BytesIO(uploaded[zip_filename]), 'r') as zip_ref:
    zip_ref.extractall("data")

print("ZIP file uploaded and extracted successfully!")


Saving HIV_AIDS_MessyDATASET.zip to HIV_AIDS_MessyDATASET (1).zip
ZIP file uploaded and extracted successfully!


In [ ]:
#importing pandas libraris

import pandas as pd
import numpy as np
import os

data_path = "data"

for file in os.listdir(data_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(data_path, file))
        print(f"{file} → Rows: {df.shape[0]}, Columns: {df.shape[1]}")

no_of_deaths_by_country_clean.csv → Rows: 510, Columns: 7
art_pediatric_coverage_by_country_clean.csv → Rows: 170, Columns: 11
no_of_cases_adults_15_to_49_by_country_clean.csv → Rows: 680, Columns: 7
art_coverage_by_country_clean.csv → Rows: 170, Columns: 11
prevention_of_mother_to_child_transmission_by_country_clean.csv → Rows: 170, Columns: 11
no_of_people_living_with_hiv_by_country_clean.csv → Rows: 680, Columns: 7


**2.1 Detection and Standardization of Missing Values**

**Purpose**



*  To identify where data is missing and understand how much information is unavailable
*   To ensure all missing values are represented in a consistent format (e.g., NaN)
*   To prevent errors during data analysis and visualization
*   To improve the accuracy and reliability of analytical results
*   To make it easier to apply missing value handling techniques such as imputation or removal
*   To avoid incorrect interpretations caused by inconsistent placeholders (e.g., "NA", "Unknown", "-", or blanks)
*   To prepare the dataset for further preprocessing and modeling steps






In [ ]:
#import required libraries

import pandas as pd
import zipfile
import io
import numpy as np

# Specify the ZIP file path
zip_path = "HIV_AIDS_MessyDATASET (2).zip"

# Create Dictionaries to store results
all_dfs = {}
missing_summary = {}

# Open the ZIP file

with zipfile.ZipFile(zip_path, 'r') as z:

# Display files inside the ZIP
    print("Files inside ZIP:")
    print(z.namelist())

#Loop through each CSV file
    for file in z.namelist():
        if file.endswith(".csv"):
            print(f"\nProcessing: {file}")

            # Read each CSV file
            df = pd.read_csv(z.open(file))

            # Standardize missing values
            df.replace(
                ['Nodata', 'No data', 'na', 'NA', ''],
                np.nan,
                inplace=True
            )

            # Store dataframe
            all_dfs[file] = df

            # Count missing values
            missing_summary[file] = df.isnull().sum()

            # Identify rows with at least one missing value
            missing_rows = df[df.isnull().any(axis=1)]

            print(f"Total rows with missing values: {len(missing_rows)}")

            # Show sample (avoid huge output)
            if not missing_rows.empty:
                print("Sample rows with missing values:")
                print(missing_rows.head(3))


Files inside ZIP:
['art_coverage_by_country_clean.csv', 'art_pediatric_coverage_by_country_clean.csv', 'no_of_cases_adults_15_to_49_by_country_clean.csv', 'no_of_deaths_by_country_clean.csv', 'no_of_people_living_with_hiv_by_country_clean.csv', 'prevention_of_mother_to_child_transmission_by_country_clean.csv']

Processing: art_coverage_by_country_clean.csv
Total rows with missing values: 34
Sample rows with missing values:
      Country Reported number of people receiving ART  \
1     Albania                                     580   
7     Austria                                     NaN   
8  Azerbaijan                                    4400   

  Estimated number of people living with HIV  \
1                                        NaN   
7                                        NaN   
8                                        NaN   

  Estimated ART coverage among people living with HIV (%)  \
1                                                NaN        
7                            

**2.2 Imputation Techniques for Numerical and Categorical Columns**

**2.2.1 Filling Missing Values in Numerical Columns Using Median**


*   The median is not affected by extreme values or outliers
*   It represents the middle value of the data, making it a reliable choice
*   It keeps the overall distribution of numerical data balanced
*   It prevents missing values from affecting calculations and analysis

**2.2.2** **Filling Missing Values in Categorical Columns Using “Unknown”**



*   It clearly marks missing or unavailable category information
*   It prevents loss of data by avoiding row deletion
*   It keeps categorical columns consistent and usable
*   It helps distinguish missing data from actual known categories







In [ ]:
# Handle numeric columns (median)

             #Identify numeric columns
            numeric_cols = df.select_dtypes(include='number').columns

            #Fill missing values in numeric columns using median
            for col in numeric_cols:
                df[col] = df[col].fillna(df[col].median())

# Handle categorical columns (Unknown)

            #Identify categorical columns
            categorical_cols = df.select_dtypes(include='object').columns

            #Fill missing values in categorical columns
            for col in categorical_cols:
                df[col] = df[col].fillna("Unknown")

            #Store the cleaned dataset
            cleaned_dfs[file] = df


            print("✔ Missing values handled safely")


✔ Missing values handled safely


**2.3 Removal of Duplicate Records from the Dataset**

**Purpose**


*   To avoid repeated records that can affect analysis results
*   To ensure each data entry is unique and accurate
*   To improve the overall quality and reliability of the dataset




In [ ]:
# Create a dictionary to store cleaned datasets
cleaned_duplicate_dfs = {}

#Loop through each cleaned dataset
for name, df in cleaned_missing_dfs.items():

    # Remove duplicate rows
    df_cleaned = df.drop_duplicates()

    # Store cleaned dataset
    cleaned_duplicate_dfs[name] = df_cleaned

print("✔ All datasets cleaned — duplicates removed.")


✔ All datasets cleaned — duplicates removed.


**2.4 Correcting data types**

**Purpose**



*   To ensure each column has the correct data type for analysis


*   To avoid errors during calculations and data processing
*   To make numerical columns usable for statistical operations


*   To maintain consistency across all datasets
*   To improve accuracy in analysis and visualizations


*   To prepare the data for further exploratory analysis



In [ ]:
# Create a dictionary to store type-corrected datasets
cleaned_types_dfs = {}

# Loop through each cleaned dataset
for name, df in cleaned_duplicate_dfs.items():

# Create a copy of the DataFrame
    df = df.copy()

    # Convert year to integer if exists
    if "year" in df.columns:
        df["year"] = df["year"].astype(int)

    # Convert numeric-like columns to numeric safely
    for col in df.columns:
        if df[col].dtype == "object":
            try:
                df[col] = pd.to_numeric(df[col])
            except:

                # If conversion fails (text column), keep as is
                pass

#Store the corrected dataset
    cleaned_types_dfs[name] = df

print(" All datasets cleaned — data types corrected.")


 All datasets cleaned — data types corrected.


In [ ]:
#to view data types of a dataset
cleaned_types_dfs["no_of_deaths_by_country_clean.csv"].dtypes


,0
Country,object
Year,int64
Count,object
Count_median,float64
Count_min,float64
Count_max,float64
WHO Region,object


**2.5** ***Create Derived Columns***

**Purpose**



*   To generate new information from existing data
*   To make analysis more meaningful and insightful
*   To simplify complex calculations

*   To highlight important patterns or relationships
*   To support better comparison and interpretation of data


*   To improve the effectiveness of visualizations

*   To prepare features for deeper analysis or modeling





**2.5.1 Derived Column: PMTCT Gap**

**Dataset**: prevention_of_mother_to_child_transmission_by_country_clean.csv

**Objective:**
In the fight against HIV/AIDS, one critical measure is the Prevention of Mother-To-Child Transmission (PMTCT). This dataset contains information about the number of pregnant women living with HIV who received antiretroviral therapy (ARVs) and the number estimated to need ARVs in each country.

**Formula**:pmtct_gap = needing_antiretrovirals_median - received_antiretrovirals

**Purpose:**

*   Quantifies the treatment gap at the country level.

*   Helps identify countries with the highest unmet needs.

*   Supports policymakers and health organizations in targeting interventions to reduce mother-to-child HIV transmission.





In [ ]:
# 1️⃣ Path to your ZIP file
zip_file_path = "HIV_AIDS_MessyDATASET (1).zip"

# 2️⃣ Open ZIP and load the PMTCT dataset
with zipfile.ZipFile(zip_file_path, 'r') as z:
    print("Files inside ZIP:", z.namelist())
    df_pmtct = pd.read_csv(z.open("prevention_of_mother_to_child_transmission_by_country_clean.csv"))

# 3️⃣ Ensure numeric columns (convert non-numeric values like "No data" to NaN)
df_pmtct["Received Antiretrovirals"] = pd.to_numeric(df_pmtct["Received Antiretrovirals"], errors="coerce")
df_pmtct["Needing antiretrovirals_median"] = pd.to_numeric(df_pmtct["Needing antiretrovirals_median"], errors="coerce")

# 4️⃣ Create derived column: pmtct_gap
df_pmtct["pmtct_gap"] = df_pmtct["Needing antiretrovirals_median"] - df_pmtct["Received Antiretrovirals"]

# 5️⃣ Show the dataset with the new column
df_pmtct.head()


Files inside ZIP: ['art_coverage_by_country_clean.csv', 'art_pediatric_coverage_by_country_clean.csv', 'no_of_cases_adults_15_to_49_by_country_clean.csv', 'no_of_deaths_by_country_clean.csv', 'no_of_people_living_with_hiv_by_country_clean.csv', 'prevention_of_mother_to_child_transmission_by_country_clean.csv']


,Country,Received Antiretrovirals,Needing antiretrovirals,Percentage Recieved,Needing antiretrovirals_median,Needing antiretrovirals_min,Needing antiretrovirals_max,Percentage Recieved_median,Percentage Recieved_min,Percentage Recieved_max,WHO Region,pmtct_gap
0,Afghanistan,20.0,200[100–500],11[7–18],200.0,100.0,500.0,11.0,7.0,18.0,Eastern Mediterranean,180.0
1,Albania,NaN,Nodata,Nodata,NaN,NaN,NaN,NaN,NaN,NaN,Europe,NaN
2,Algeria,320.0,500[500–500],74[69–78],500.0,500.0,500.0,74.0,69.0,78.0,Africa,180.0
3,Angola,9600.0,25000[19000–32000],38[29–48],25000.0,19000.0,32000.0,38.0,29.0,48.0,Africa,15400.0
4,Argentina,1800.0,1800[1600–2000],95[85–95],1800.0,1600.0,2000.0,95.0,85.0,95.0,Americas,0.0


**2.5.2 Derived Column**: PMTCT Coverage Ratio

**Objective**: pmtct_coverage_ratio represents the fraction of pregnant women living with HIV who actually received antiretroviral therapy (ARVs) out of those estimated to need it. It shows how much of the target population is being covered.

**Formula**:pmtct_coverage_ratio = received_antiretrovirals / needing_antiretrovirals_median

**Purpose**: This column helps to measure treatment coverage in each country. A higher ratio indicates better ARV coverage for preventing mother-to-child transmission, while a lower ratio highlights countries that need more attention and resources.

In [ ]:
# 1. Path to your ZIP file
zip_file_path = "HIV_AIDS_MessyDATASET (1).zip"

# 2️. Open ZIP and load the PMTCT dataset
with zipfile.ZipFile(zip_file_path, 'r') as z:
    print("Files inside ZIP:", z.namelist())
    df_pmtct = pd.read_csv(z.open("prevention_of_mother_to_child_transmission_by_country_clean.csv"))

# 3. Ensure numeric columns (convert non-numeric values like "No data" to NaN)
df_pmtct["Received Antiretrovirals"] = pd.to_numeric(df_pmtct["Received Antiretrovirals"], errors="coerce")
df_pmtct["Needing antiretrovirals_median"] = pd.to_numeric(df_pmtct["Needing antiretrovirals_median"], errors="coerce")

# 4. Create derived column: PMTCT Coverage Ratio
df_pmtct["pmtct_coverage_ratio"] = df_pmtct["Received Antiretrovirals"] / df_pmtct["Needing antiretrovirals_median"]

# 5. Show the dataset with the new columns
df_pmtct.head()

Files inside ZIP: ['art_coverage_by_country_clean.csv', 'art_pediatric_coverage_by_country_clean.csv', 'no_of_cases_adults_15_to_49_by_country_clean.csv', 'no_of_deaths_by_country_clean.csv', 'no_of_people_living_with_hiv_by_country_clean.csv', 'prevention_of_mother_to_child_transmission_by_country_clean.csv']


,Country,Received Antiretrovirals,Needing antiretrovirals,Percentage Recieved,Needing antiretrovirals_median,Needing antiretrovirals_min,Needing antiretrovirals_max,Percentage Recieved_median,Percentage Recieved_min,Percentage Recieved_max,WHO Region,pmtct_coverage_ratio
0,Afghanistan,20.0,200[100–500],11[7–18],200.0,100.0,500.0,11.0,7.0,18.0,Eastern Mediterranean,0.100
1,Albania,NaN,Nodata,Nodata,NaN,NaN,NaN,NaN,NaN,NaN,Europe,NaN
2,Algeria,320.0,500[500–500],74[69–78],500.0,500.0,500.0,74.0,69.0,78.0,Africa,0.640
3,Angola,9600.0,25000[19000–32000],38[29–48],25000.0,19000.0,32000.0,38.0,29.0,48.0,Africa,0.384
4,Argentina,1800.0,1800[1600–2000],95[85–95],1800.0,1600.0,2000.0,95.0,85.0,95.0,Americas,1.000


***2.6 Filtering and Aggregation***



**Objective:**
The goal of this step is to focus on specific subsets of the data (filtering) and summarize key metrics (aggregation) to gain meaningful insights about HIV/AIDS across countries and regions.




**2.6.1 Filtering:**

Selecting only the rows that meet certain conditions, such as a specific WHO region or year.

**Example** : Filter the dataset to include only African countries in the year 2018.

**Purpose:** Helps to analyze targeted data and avoid unnecessary information from other regions or years.

In [ ]:
# 1️⃣ Load dataset from ZIP
zip_file_path = "HIV_AIDS_MessyDATASET (1).zip"

# Choose the dataset you want to filter
dataset_name = "no_of_people_living_with_hiv_by_country_clean.csv"

with zipfile.ZipFile(zip_file_path, 'r') as z:
    df = pd.read_csv(z.open(dataset_name))

# 2️⃣ Clean column names (remove spaces)
df.columns = df.columns.str.strip()

# Filter by WHO region
df_filtered_region = df[df['WHO Region'] == 'Africa']
print("Filtered by WHO Region (Africa):")
print(df_filtered_region.head())


Filtered by WHO Region (Africa):
         Country  Year                  Count  Count_median  Count_min  \
2        Algeria  2018     16000[15000–17000]       16000.0    15000.0   
3         Angola  2018  330000[290000–390000]      330000.0   290000.0   
16         Benin  2018    73000[48000–120000]       73000.0    48000.0   
20      Botswana  2018  370000[330000–400000]      370000.0   330000.0   
24  Burkina Faso  2018    96000[78000–120000]       96000.0    78000.0   

    Count_max WHO Region  
2     17000.0     Africa  
3    390000.0     Africa  
16   120000.0     Africa  
20   400000.0     Africa  
24   120000.0     Africa  


**2.6.2 Aggregation**:

Summarizing data by calculating metrics like sum, mean, or median for groups.

**Example**: Calculate the total number of people living with HIV for each WHO region.

**Purpose**: Provides a concise overview of trends and totals, making it easier to compare regions or countries and identify patterns.



In [ ]:
# Load dataset from ZIP
zip_file_path = "HIV_AIDS_MessyDATASET (1).zip"
with zipfile.ZipFile(zip_file_path, 'r') as z:
    df = pd.read_csv(z.open("no_of_people_living_with_hiv_by_country_clean.csv"))

# Strip column names (remove any spaces)
df.columns = df.columns.str.strip()

# Ensure numeric column and convert errors to NaN
df['Count_median'] = pd.to_numeric(df['Count_median'], errors='coerce')

# Drop rows where count_median is NaN
df = df.dropna(subset=['Count_median'])

# Aggregation: total people living with HIV by WHO region
df_agg = df.groupby('WHO Region')['Count_median'].sum().reset_index()
df_agg.rename(columns={'Count_median': 'total_people_living_with_hiv'}, inplace=True)

print("\nAggregated total people living with HIV by WHO region:")
print(df_agg)



Aggregated total people living with HIV by WHO region:
              WHO Region  total_people_living_with_hiv
0                 Africa                    83849000.0
1               Americas                     8073100.0
2  Eastern Mediterranean                      826070.0
3                 Europe                     3281540.0
4        South-East Asia                     4920170.0
5        Western Pacific                     1793000.0


**2.7 Saving and Downloading Cleaned Datasets as a ZIP Archive**

In [ ]:
#Create a folder to store cleaned files
cleaned_folder = "cleaned_data"
os.makedirs(cleaned_folder, exist_ok=True)

# Write cleaned CSV files correctly
for name, df in cleaned_dfs.items():
    cleaned_path = os.path.join(cleaned_folder, name)
    df.to_csv(cleaned_path, index=False)

# Create ZIP after all files exist
cleaned_zip_name = "HIV_AIDS_CleanedDATASET.zip"

#Create a ZIP archive
with zipfile.ZipFile(cleaned_zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:

#Add cleaned CSV files to the ZIP
    for file in os.listdir(cleaned_folder):
        file_path = os.path.join(cleaned_folder, file)
        if os.path.isfile(file_path):
            zipf.write(file_path, arcname=file)

# Download ZIP
files.download(cleaned_zip_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>